In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[4],3b
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,0
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[5, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  76.911574                    1  54.636205
0.000750        2  41.020533                    2  41.629703
0.001052        3  33.259869                    3  36.060282
0.001476        4  26.397425                    4  29.531372
0.002070        5  20.664232                    5  22.680055
0.002904        6  16.081310                    6  16.382545
0.004074        7  12.530285                    7  11.549865
0.005714        8   9.827018                    8   8.492917
0.008015        9   7.770893                    9   6.801298
0.011243       10   6.180803                   10   5.730593
0.015771       11   4.903649                   11   4.712805
0.022122       12   3.810097                   12   3.608425
0.031031       13   2.745222                   13   2.392883
0.043528       14   1.399978                   14   0.992763
0.061057       15   0.979141                   15   0.634240
0.085645       16   1.752999                   16   1.560913
0.120136       17   2.555488                   17   2.559920
0.168516       18   3.176323                   18   3.352582
0.236378       19   3.831257                   19   4.046337
0.331549       20   4.628079                   20   4.670991
0.465100       21   5.780301                   21   5.568341
0.652400       22   7.472781                   22   7.159129
0.915100       23   7.603271                   23   7.459041
1.283650       24   6.451719                   24   6.408524
1.800600       25   5.474882                   25   5.300224
2.525700       26   4.427503                   26   4.162870
3.542800       27   3.636540                   27   3.592638
4.969550       28   3.059310                   28   3.192447
6.970850       29   2.591584                   29   2.693923
9.778100       30   2.179984                   30   2.197782
13.715850      31   1.799270                   31   1.826057
19.239350      32   1.443215                   32   1.415439
26.987250      33   1.121664                   33   1.091365
37.855300      34   0.874324                   34   0.933570
53.100050      35   0.507275                   35   0.568150
73.887500      36   0.034923                   36   0.021046
97.662500      37  -0.289156                   37  -0.323061
121.437500     38  -0.219236                   38  -0.220491
145.212500     39   0.020826                   39   0.036257
168.987500     40   0.075532                   40   0.082616
192.762500     41   0.099899                   41   0.102419
216.537500     42   0.111780                   42   0.112123
240.312500     43   0.116942                   43   0.114397
264.087500     44   0.118045                   44   0.111346
287.862500     45   0.116553                   45   0.105641
311.637500     46   0.113290                   46   0.099258
335.412500     47   0.108849                   47   0.092935
359.187500     48   0.103653                   48   0.088446
382.962500     49   0.097977                   49   0.084017
406.737500     50   0.092141                   50   0.080868
430.512500     51   0.086304                   51   0.078446
454.287500     52   0.080623                   52   0.074806
478.062500     53   0.075138                   53   0.072659
501.837500     54   0.069950                   54   0.068529
525.612500     55   0.065174                   55   0.066545
549.387500     56   0.060815                   56   0.062177
573.162500     57   0.057038                   57   0.059212
596.937500     58   0.054224                   58   0.055943
620.712500     59   0.052987                   59   0.054271
644.487500     60   0.054949                   60   0.055786
668.262500     61   0.061355                   61   0.061656
692.037500     62   0.069008                   62   0.068748
715.812500     63   0.067517                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -16.254068   0.000000 -16.254068 -15.835989 -1.311260e-07   
0.000624    2     -16.254436   0.006056 -16.248379 -15.836385  4.437928e-03   
0.000876    3     -16.254603   0.007445 -16.247157 -15.836551  5.844288e-03   
0.001229    4     -16.254812   0.009044 -16.245768 -15.836764  7.563973e-03   
0.001723    5     -16.255074   0.010854 -16.244221 -15.837038  9.568562e-03   
0.002417    6     -16.255405   0.012883 -16.242522 -15.837389  1.178489e-02   
0.003391    7     -16.255824   0.015157 -16.240667 -15.837845  1.412975e-02   
0.004757    8     -16.256362   0.017721 -16.238640 -15.838437  1.659043e-02   
0.006672    9     -16.257057   0.020647 -16.236411 -15.839212  1.929346e-02   
0.009359    10    -16.257970   0.024033 -16.233937 -15.840240  2.248579e-02   
0.013128    11    -16.259189   0.028011 -16.231178 -15.841615  2.642011e-02   
0.018415    12    -16.260851   0.032745 -16.228107 -15.843498  3.125426e-02   
0.025830    13    -16.263185   0.038425 -16.224760 -15.846128  3.705484e-02   
0.036232    14    -16.266652   0.045275 -16.221377 -15.849993  4.386809e-02   
0.050823    15    -16.272641   0.053684 -16.218958 -15.856426  5.201698e-02   
0.071291    16    -16.282331   0.065747 -16.216584 -15.866551  6.368052e-02   
0.100000    17    -16.295514   0.084892 -16.210622 -15.880189  8.262751e-02   
0.140271    18    -16.312196   0.113765 -16.198431 -15.897298  1.119495e-01   
0.196760    19    -16.332902   0.155726 -16.177177 -15.918265  1.553527e-01   
0.275997    20    -16.357731   0.216515 -16.141216 -15.942997  2.180680e-01   
0.387100    21    -16.385270   0.304963 -16.080306 -15.969916  3.064683e-01   
0.543100    22    -16.408114   0.434624 -15.973490 -15.991791  4.312524e-01   
0.761700    23    -16.399970   0.619986 -15.779985 -15.983624  6.084886e-01   
1.068500    24    -16.348487   0.844825 -15.503661 -15.933632  8.296062e-01   
1.498800    25    -16.263914   1.089110 -15.174804 -15.851964  1.074628e+00   
2.102400    26    -16.142986   1.359640 -14.783346 -15.734984  1.336656e+00   
2.949000    27    -15.993169   1.653839 -14.339330 -15.589418  1.608611e+00   
4.136600    28    -15.814574   1.986831 -13.827743 -15.413519  1.938175e+00   
5.802500    29    -15.601602   2.377576 -13.224026 -15.202464  2.357175e+00   
8.139200    30    -15.348183   2.841504 -12.506678 -14.947989  2.848452e+00   
11.417000   31    -15.048074   3.387837 -11.660237 -14.643612  3.397516e+00   
16.014700   32    -14.695350   4.015049 -10.680300 -14.282159  4.030692e+00   
22.464000   33    -14.284245   4.706511  -9.577734 -13.860863  4.690854e+00   
31.510500   34    -13.800127   5.424394  -8.375733 -13.363249  5.362892e+00   
44.200100   35    -13.154004   6.092532  -7.061472 -12.700834  6.103940e+00   
62.000000   36    -12.436785   6.444915  -5.991870 -11.969161  6.570350e+00   
85.775000   37    -12.199267   6.305751  -5.893515 -11.725776  6.386246e+00   
109.550000  38    -12.862807   6.154936  -6.707871 -12.430968  6.181502e+00   
133.325000  39    -14.076385   6.751076  -7.325309 -13.710903  6.840398e+00   
157.100000  40    -15.303670   8.037015  -7.266656 -14.992942  8.224560e+00   
180.875000  41    -16.528801   9.474867  -7.053934 -16.262217  9.726533e+00   
204.650000  42    -17.745588  10.973001  -6.772587 -17.512914  1.126570e+01   
228.425000  43    -18.950082  12.492304  -6.457778 -18.746532  1.281513e+01   
252.200000  44    -20.139331  14.010899  -6.128432 -19.959159  1.434996e+01   
275.975000  45    -21.311691  15.515712  -5.795979 -21.151374  1.585580e+01   
299.750000  46    -22.466145  16.998416  -5.467729 -22.323407  1.732538e+01   
323.525000  47    -23.602153  18.453483  -5.148670 -23.472351  1.875389e+01   
347.300000  48    -24.719490  19.877374  -4.842116 -24.602256  2.014556e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')